In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
i_food=pd.read_csv("../input/indian-food-101/indian_food.csv")

In [ ]:
i_food.head()

In [ ]:
print(i_food['name'].isna().sum())
print(i_food['diet'].isna().sum())
print(i_food['prep_time'].isna().sum())
print(i_food['cook_time'].isna().sum())

print(i_food['flavor_profile'].isna().sum())
print(i_food['course'].isna().sum())
print(i_food['state'].isna().sum())
print(i_food['region'].isna().sum())


There is a null value in Region column.
The corresponding value in State is 'Uttar Pradesh' hence we can impute it as 'North'

In [ ]:
i_food['region'].fillna('North',inplace=True)

We can start the analysis right away.
I am going to analyse this from the perspective of a Online Restaurant Owner or a Startup.

For this I need to know 
1)Geographic popularity of flavor, course.
2)Regional distribution of diet (Veg/Non Veg).
3)Average cooking time of fastest/slowest foods(This is to estimate the prep and delivery time).
4)Availibity of food grains in the region nearby. This so we can calculate transportation costs.


Diet type distribution

In [ ]:
print(i_food['diet'].unique())
print(i_food['diet'].value_counts())

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.axis('equal')
diets = ['Vegetarian','Non_Vegtarian']
nums = [226,29]
ax.pie(nums, labels = diets,autopct='%1.2f%%')
plt.show()

Unsurprisingly 88.63% share is of Vegetarians.
India a deeply spiritual nation, with a Hindu Majority(79.8%) this distribution is reassuring.

From business point of view larger amount of money needs to go in to Veg types.


In [ ]:
i_reg=i_food.loc[i_food['region']!='-1']
i_reg=i_reg.loc[i_reg['flavor_profile']!='-1']


In [ ]:
print(i_reg['region'].unique())
print(i_reg['flavor_profile'].unique())

We got here six different regions and four different types of flavors.
Lets see geographic distribution of flavors region-wise.

In [ ]:
i_reg['flavor_profile'].value_counts()

Abvoe table is an important observation, 
Spicy and Sweet are the most sought dishes.
As an Indian charcteristic 'spicy' tops the chart.

In [ ]:
i_reg[['flavor_profile','prep_time','cook_time']].groupby('flavor_profile').sum()

The top dishes spicy and sweet  have comparable cook_times (No issues here).
Preparation time for sweet dishes is lot lesser (just more than half of spicy), this feature can be advertised more as it is early to deliver.


In [ ]:
import seaborn as sns


In [ ]:
rw=2
col=2
cnt=1
fig = plt.figure(figsize=(20,14))


for flvr in (i_reg['flavor_profile'].unique()):
    flvr_df=i_reg.loc[i_reg['flavor_profile']==flvr]
    flvr_unq=flvr_df['region']
    flvr_unq=flvr_unq.value_counts()
    flvr_unq=flvr_unq.to_frame()
    flvr_unq=flvr_unq.reset_index()
    
    flvr_unq['percent']=(flvr_unq['region']/flvr_unq['region'].sum())*100
    
    
    plt.subplot(rw,col,cnt)
        
    g=sns.barplot(x='index',y='percent',data=flvr_unq)
    g.set_title(flvr.upper(), fontsize=16)

    for index, row in flvr_unq.iterrows():
        g.text(row.name,row.percent,round(row.percent,2), color='black', ha="center", fontsize=12)
        
    
        
    cnt=cnt+1


Lets examine the above charts.
1)Western part of India (Maharashtra, Gujarat) have a versatile taste for the cuisines. This region tops all thecharts.

2)Second to this is the Northern region, Sweet dishes are not so popular here.
This may be due to the fact the it is a thickly populated region of India. As much as 45% of population is concentrated around Ganga-river basin.
Population pressure may deter '**affordability**'


3)North East has lower percentages in all of its charts, this again because of lower population density and also thick forestry.
Tribal population may not have a taste of Urban cuisines.

4)Sweet dishes are very popular in Eastern region. This so because of states like Bihar and West Bengal where sweet dishes like rasogolla, rasmalai are much sought.

5)Sugarcance is the most sought sweetener. In the West it is Maharashtra, in the North is Uttar Pradesh which can also cater to eastern region as well. With abundant sugarcane producing states nearby, raw material availibility should not be a problem.


AS the overall picture is known. Now we can focus on two most popular types sweet and spicy for deeper observation.

In [ ]:
i_spicy_sweet=i_food[['name','ingredients','flavor_profile', 'course']]
i_spicy=i_spicy_sweet.loc[i_spicy_sweet['flavor_profile']=='spicy']
i_sweet=i_spicy_sweet.loc[i_spicy_sweet['flavor_profile']=='sweet']

print( "SPICY \n\n", i_spicy['course'].value_counts())
print("\n")
print("SWEET\n\n",i_sweet['course'].value_counts())


In the main course SPICY flavor is most preferred.
DESSERTS are mainly sweet

This is an unsurprising observation though.

Next we can use word cloud method to get idea, on frequency of raw materials used

In [ ]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS 

#WE use stopwords to ignore obvious words, they add no value to our quality of info.

txt_spicy = " ".join(ing for ing in i_spicy.ingredients)

wordcloud = WordCloud(stopwords=['flour','dal','masala','seeds','green','powder','oil'],background_color="white",width=400,height=300).generate(txt_spicy)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Popular Spice ingredients are Garam Masala, curry leaves, mustard seeds, ginger, Chana, peas, chillis.
Rice, coconut(chutney), urad dal are the main food items along with which these spicy supplements are consumed.

In [ ]:
txt_sweet = " ".join(ing for ing in i_sweet.ingredients)

wordcloud = WordCloud(stopwords=['flour','sugar'],background_color="white",width=400,height=300).generate(txt_sweet)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


Among sweets milk (ofcourse) and ghee are much sought after.
These two are diary products. One more important point here is Milk is the largest crop in India and globally India is the largest procuder of milk.

Though Uttar Pradesh is the largest milk producer, Northern region is not a predominant user of it.
In the West major producing states are Gujarat(home place of Amul) and Mahasrashtra.
In the East Bihar and West Bengal feature.
Though, these states lag behind compared to UP by a large margin.

This shall add to transportation and storage(cold chain) costs for West.

Sugar(default choice), despite that jaggery is also preferred.
Jaggery is preprocessed sugar. Organic users prefer  jaggery over sugar.
In south dished like pongal are made of jaggery.

Rice and coconut are most preferred main ingredients along with which these sweet making supplements are consumed.